In [1]:
from pathlib import Path
API_PATH = Path("/Users/joregan/Playing/rdapi/api_output")

In [3]:
from sync_asr.riksdag.riksdag_api import RiksdagAPI

In [31]:
SAMPLE = RiksdagAPI(filename=str(API_PATH / "H2C120150122fs"))

In [32]:
YEARS_RAW = {}
YEARS_TRANSCRIBED = {}

In [38]:
import re

def sanity_check(api_file, key):
    if "videodata" not in api_file.__dict__:
        return False
    if key not in api_file.__dict__["videodata"]:
        return False
    else:
        return True


def get_year(api_file):
    if not sanity_check(api_file, "debatedate"):
        return None
    year = re.search("([12][0-9][0-9][0-9])$", api_file.__dict__["videodata"]["debatedate"].strip())
    if year:
        return year.group(0)
    else:
        return None

In [43]:
def get_raw_seconds(api_file):
    if not sanity_check(api_file, "debateseconds"):
        return 0
    return api_file.__dict__["videodata"]["debateseconds"]


In [53]:
type(get_raw_seconds(SAMPLE))

int

In [56]:
def get_transcribed_total(rdapi):
    total = 0
    if not sanity_check(SAMPLE, "speakers"):
        return total
    for speaker in rdapi.__dict__["videodata"]["speakers"]:
        if "duration" in speaker:
            total += speaker["duration"]
    return total

In [57]:
for filename in API_PATH.glob("*"):
    api_file = RiksdagAPI(filename=str(API_PATH / filename))
    year = get_year(api_file)
    if year is None:
        continue
    raw = get_raw_seconds(api_file)
    ts = get_transcribed_total(api_file)
    if year in YEARS_RAW:
        YEARS_RAW[year] += raw
    else:
        YEARS_RAW[year] = raw
    if year in YEARS_TRANSCRIBED:
        YEARS_TRANSCRIBED[year] += ts
    else:
        YEARS_TRANSCRIBED[year] = ts

KeyError: 'speakers'